# FX2LP custom I2C bridge test

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

nano 70-ch341.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout"
SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", MODE="666"

then reloaded udev rules:

$ sudo udevadm control --reload-rules  
$ sudo udevadm trigger

```

In [1]:
%pylab inline  

import usb
from array import array

Populating the interactive namespace from numpy and matplotlib


## Backend

## Device

In [2]:
# import usb.backend.libusb1 as libusb
import usb.backend.libusb0 as libusb 

device = usb.core.find(idVendor = 0X04B4, idProduct = 0X1004, backend = libusb.get_backend())
device.set_configuration()

# print(device)

In [3]:
bmRequest_Type_vendor_class_read = 0xC0
bmRequest_Type_vendor_class_write = 0x40

VR_DOWNLOAD = 0x22
VR_UPLOAD = 0xa2  #  
VR_RENUM = 0xa3  # renum
VR_I2C_RATE = 0xa4  # I2C bus rate, 0: 100KHz, non-zero: 400 KHz

### I2C Read / Write

In [4]:
I2C_address = 0x60
register_address = 3

In [5]:
def read_byte(I2C_address, register_address):
    
    return device.ctrl_transfer(bmRequestType = bmRequest_Type_vendor_class_read,
                                bRequest = VR_UPLOAD, 
                                wValue = I2C_address  ,
                                wIndex = register_address  ,
                                data_or_wLength = 1)[0]

read_byte(I2C_address = I2C_address, register_address = register_address)

255

In [6]:
def write_byte(I2C_address, register_address, value):
    
    return device.ctrl_transfer(bmRequestType = bmRequest_Type_vendor_class_write,
                                bRequest = VR_DOWNLOAD, 
                                wValue = I2C_address,
                                wIndex = register_address,
                                data_or_wLength = array('B', [value])) 

write_byte(I2C_address = I2C_address, register_address = register_address, value = 0xfd)

1

In [7]:
read_byte(I2C_address = I2C_address, register_address = register_address)

253

In [8]:
for i in range(188):
    try:
        v = read_byte(0x60, i)
        print(i, v)
    except:
        pass

0 17
1 248
2 3
3 253
4 8
5 255
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 144
15 0
16 0
17 0
18 0
19 128
20 128
21 128
22 128
23 128
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 1
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 255
177 12
178 0
179 0
180 0
18

### I2C Speed

In [9]:
# 100KHz
def set_i2c_speed(as_400KHz = True):
    device.ctrl_transfer(bmRequestType = bmRequest_Type_vendor_class_write, bRequest = VR_I2C_RATE, wValue = int(bool(as_400KHz))) 

In [10]:
set_i2c_speed(False)

In [11]:
set_i2c_speed(True)

### Re-Numerate

In [12]:
def re_numerate():
    device.ctrl_transfer(bmRequestType = bmRequest_Type_vendor_class_write, bRequest = VR_RENUM) 